### Ingestion del archivo "language.csv"

In [0]:
dbutils.widgets.text("p_environment", "")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer el archivo CSV usando "DataFrameReader" de Spark

In [0]:
# importamos los data types
from pyspark.sql.types import *

In [0]:
# definimos el schema
language_schema = StructType( fields= [
    StructField("languageId", IntegerType(), False),
    StructField("languageCode", StringType(), True),
    StructField("languageName", StringType(), True)
])

In [0]:
language_df = spark.read \
    .option("header", True) \
    .schema(language_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/language.csv")

In [0]:
display(language_df)

In [0]:
display(dbutils.fs.mounts())

In [0]:
%fs
ls /mnt/moviehistory2025/bronze

#### Paso 2 - Seleccionamos solo las columnas requeridas

In [0]:
# forma 1
languages_selected_df = language_df.select("languageId", "languageName")

#### Paso 3 - Cambiar el nombre de las columnas segun lo requerido

In [0]:
# forma 1
languages_renamed_df = languages_selected_df \
                    .withColumnRenamed("languageId", "language_id") \
                    .withColumnRenamed('languageName', 'language_name')

#### Paso 4 - Agregar la columna "ingestion_date" y "environment" al DF

In [0]:
from pyspark.sql.functions import current_timestamp, lit

# forma 1
languages_final_df = add_ingestion_date(languages_renamed_df) \
                                   .withColumn("environment", lit(v_environment)) \
                                    .withColumn("file_date", lit(v_file_date))

#### Paso 5 - Escribir datos en el datalake en formato "parquet"

In [0]:
languages_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages")

In [0]:
%sql
select * from movie_silver.languages

In [0]:
display(spark.read.format("delta").load("/mnt/moviehistory2025/silver/languages"))

In [0]:
%sql
SELECT * FROM movie_silver.languages

In [0]:
dbutils.notebook.exit("success")